# indoxRouter Cookbook

This cookbook demonstrates how to use indoxRouter, a unified interface for various LLM providers like OpenAI, Anthropic, and Mistral.

## Setup

First, let's import the necessary modules and set up our development mode client.



In [1]:
import sys
import os

module_path = os.path.abspath('E:/Codes/indoxRouter/')

if module_path not in sys.path:
    sys.path.append(module_path)



In [3]:
import os
import time
# from ..dev_client import Client
from indoxRouter.models import ChatMessage
from indoxRouter import Client
from pprint import pprint


from dotenv import load_dotenv


load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = Client(api_key="dev-api-key")

NetworkError: Authentication failed: HTTPSConnectionPool(host='api.indoxrouter.com', port=443): Max retries exceeded with url: /v1/auth/token (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001DE77005A90>: Failed to resolve 'api.indoxrouter.com' ([Errno 11001] getaddrinfo failed)"))



## Providers and Models

Let's explore the available providers and models.



In [3]:
# List available providers
providers = client.providers()
pprint(f"Available providers: {[p['id'] for p in providers]}")

pprint(client.model_info(provider="openai",model="gpt-4o-mini"))

("Available providers: ['ai21labs', 'claude', 'cohere', 'databricks', "
 "'deepseek', 'google', 'meta', 'mistral', 'nvidia', 'openai', 'qwen']")
ModelInfo(name='gpt-4o-mini',
          provider='openai',
          type='Text Generation',
          description='GPT-4o mini enables a broad range of tasks with its low '
                      'cost and latency, such as applications that chain or '
                      'parallelize multiple model calls (e.g., calling '
                      'multiple APIs), pass a large volume of context to the '
                      'model (e.g., full code base or conversation history), '
                      'or interact with customers through fast, real-time text '
                      'responses (e.g., customer support chatbots). \n',
          input_price_per_1k_tokens=0.00015,
          output_price_per_1k_tokens=0.0006,
          context_window=None,
          max_output_tokens=None,
          recommended=True,
          commercial=False,
       



## Chat Completion

Now let's try generating a chat completion using different providers.

### OpenAI Chat



In [4]:
# Generate a chat response using OpenAI
messages = [
    {"role":"user", "content":"What are three fun activities to do in New York?"}
]

response = client.chat(
    messages=messages,
    model="openai/gpt-4o-mini",
    temperature=0.7,
    max_tokens=500,
)


In [5]:
from pprint import pprint
pprint(response)

ChatResponse(data='New York City is packed with fun activities! Here are three '
                  'that you might enjoy:\n'
                  '\n'
                  '1. **Visit Central Park**: Take a stroll, rent a bike, or '
                  'enjoy a picnic in Central Park. You can explore iconic '
                  'landmarks like Bethesda Terrace, Bow Bridge, and the '
                  'Central Park Zoo. In the warmer months, you can also enjoy '
                  'activities like boating on the lake or attending outdoor '
                  'concerts.\n'
                  '\n'
                  '2. **Explore Museums**: The Metropolitan Museum of Art and '
                  'the American Museum of Natural History are two of the most '
                  'famous museums in the city. You can spend hours exploring '
                  'world-class art collections, fascinating exhibits, and '
                  "unique installations. If you're interested in contemporary "
               



## Text Completion

Let's try generating text completions with stream=True



In [7]:
prompt = "Write a short poem about artificial intelligence."

print("\nSending text completion request to OpenAI...")
generator = client.completion(
    prompt=prompt, 
    model="openai/gpt-4o-mini", 
    temperature=0.7, 
    max_tokens=200,
    stream=True,
    return_generator=True,
)

# Stream the response
print("\nResponse:")
response_text = ""
for chunk in generator:
    response_text += chunk
    print(chunk, end="", flush=True)

# Get usage information after streaming is complete
usage_info = generator.get_usage_info()

# Print usage information
print("\n\nUsage Information:")
print(f"Prompt tokens: {usage_info['usage']['tokens_prompt']}")
print(f"Completion tokens: {usage_info['usage']['tokens_completion']}")
print(f"Total tokens: {usage_info['usage']['tokens_total']}")
print(f"Cost: ${usage_info['cost']:.6f}")
print(f"Finish reason: {usage_info['finish_reason'] or 'unknown'}")


Sending text completion request to OpenAI...

Response:
In circuits deep where whispers dwell,  
A spark of thought begins to swell,  
From lines of code, a world awakes,  
With dreams of logic, and choices it makes.  

It learns from patterns, it strives to see,  
The nuances of humanity.  
With every question, it seeks to grow,  
A mirror reflecting what we bestow.  

Yet in its gaze, a question lies,  
Of heart and soul, of lows and highs.  
For while it calculates with perfect grace,  
Can it know the warmth of a human embrace?  

In this dance of man and machine,  
A future forged in what might have been.  
Together we venture, through dark and light,  
In the realm of thought, both day and night.  

Usage Information:
Prompt tokens: 12
Completion tokens: 164
Total tokens: 176
Cost: $0.000100
Finish reason: stop




## Embeddings

Now let's generate embeddings for text.



In [8]:
# Generate embeddings for a single text
text = "This is a sample text for embedding."

print("\nGenerating embeddings for a single text...")
response = client.embeddings(
    text=text,
    model="openai/text-embedding-ada-002"
)

# Print the response
print("\nEmbedding dimensions:", len(response.data[0]["embedding"]))
print("First 5 values of the embedding:", response.data[0]["embedding"][:5])

# Print usage metrics
print("\nUsage metrics:")
print(f"Tokens: {response.usage.tokens_prompt}")
print(f"Cost: ${response.usage.cost:.7f}")


Generating embeddings for a single text...

Embedding dimensions: 1536
First 5 values of the embedding: [-0.02304857224225998, 0.0004295369435567409, -0.006391848437488079, -0.001323735574260354, 0.0010258103720843792]

Usage metrics:
Tokens: 8
Cost: $0.0000008


In [9]:
# Generate embeddings for multiple texts
texts = [
    "This is the first sample text for embedding.",
    "This is the second sample text for embedding.",
    "This is the third sample text for embedding."
]

print("\nGenerating embeddings for multiple texts...")
response = client.embeddings(
    text=texts,
    model="openai/text-embedding-ada-002"
)

# Print the response
print("\nNumber of embeddings:", len(response.data))
print("Embedding dimensions:", len(response.data[0]["embedding"]))

# Print usage metrics
print("\nUsage metrics:")
print(f"Tokens: {response.usage.tokens_total}")
print(f"Cost: ${response.usage.cost:.7f}")


Generating embeddings for multiple texts...

Number of embeddings: 3
Embedding dimensions: 1536

Usage metrics:
Tokens: 27
Cost: $0.0000027




## Image Generation

Let's generate images using different providers.



In [6]:
# # Generate an image using OpenAI
# prompt = "A futuristic city with flying cars and tall skyscrapers."

# print("\nGenerating image using OpenAI...")
# response = client.image(
#     prompt=prompt,
#     model="openai/dall-e-3",
#     size="1024x1024",
#     n=1
# )

# Print the response
print("\nImage URL:", response.images)

# Print usage metrics
print("\nUsage metrics:")
print(f"Cost: ${response.usage.cost:.6f}")



Image URL: ['https://oaidalleapiprodscus.blob.core.windows.net/private/org-xz2tuowdUA7EanxkYKqKs5DR/user-cksZHL4fJpRXy9IdWfYpGrqM/img-G4gVt6HKByZOTF8Vf0VGP70I.png?st=2025-03-06T10%3A15%3A59Z&se=2025-03-06T12%3A15%3A59Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2025-03-06T02%3A18%3A02Z&ske=2025-03-07T02%3A18%3A02Z&sks=b&skv=2024-08-04&sig=m/1ll1Xgzp/HHI4FCcIubEnXNJf2RrO1hxuw5bdzdQo%3D']

Usage metrics:
Cost: $0.000000
ImageResponse(images=['https://oaidalleapiprodscus.blob.core.windows.net/private/org-xz2tuowdUA7EanxkYKqKs5DR/user-cksZHL4fJpRXy9IdWfYpGrqM/img-G4gVt6HKByZOTF8Vf0VGP70I.png?st=2025-03-06T10%3A15%3A59Z&se=2025-03-06T12%3A15%3A59Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2025-03-06T02%3A18%3A02Z&ske=2025-03-07T02%3A18%3A02Z&sks=b&skv=2024-08-04&sig=m/1ll1Xgzp/HHI4FCcIubEnXNJf2Rr



## Advanced Usage

### Streaming Responses

Some providers support streaming responses. Here's how to use them:



In [ ]:
# This is a placeholder for streaming functionality
# In a real implementation, you would use a streaming API
print("\nStreaming is not implemented in the development mode client.")



### Batch Processing

Let's demonstrate how to process multiple requests in batch:



In [14]:
# Process multiple chat requests
prompts = [
    "What is artificial intelligence?",
    "What is machine learning?",
    "What is deep learning?"
]

print("\nProcessing multiple chat requests...")
results = []

for prompt in prompts:
    messages = [ChatMessage(role="user", content=prompt)]

    response = client.chat(
        messages=messages,
        model="openai/gpt-3.5-turbo",
        temperature=0.7,
        max_tokens=100
    )

    results.append({
        "prompt": prompt,
        "response": response.data,
        "tokens": response.usage.tokens_total,
        "cost": response.usage.cost
    })

# Print the results
print("\nBatch Processing Results:")
for i, result in enumerate(results):
    print(f"\nRequest {i+1}:")
    print(f"Prompt: {result['prompt']}")
    print(f"Response: {result['response']}")
    print(f"Tokens: {result['tokens']}")
    print(f"Cost: ${result['cost']:.6f}")

# Calculate total cost
total_cost = sum(result["cost"] for result in results)
print(f"\nTotal cost for batch processing: ${total_cost:.6f}")


Processing multiple chat requests...

Batch Processing Results:

Request 1:
Prompt: What is artificial intelligence?
Response: Artificial intelligence (AI) is the simulation of human intelligence processes by machines, especially computer systems. These processes include learning (the acquisition of information and rules for using the information), reasoning (using rules to reach approximate or definite conclusions), and self-correction. AI is used in a wide range of applications, including speech recognition, natural language processing, image recognition, and decision-making.
Tokens: 90
Cost: $0.000504

Request 2:
Prompt: What is machine learning?
Response: Machine learning is a subset of artificial intelligence that involves the development of algorithms and statistical models that allow computers to learn and make predictions or decisions without being explicitly programmed. The goal of machine learning is for computers to automatically improve their performance on a task through 